# Collaborative Filtering
- Mean regularization
- 텐서플로우 GradientTape
    - AutoDiff: 자동으로 도함수를 취한다
    - AutoGrad: 위 기능 수행하는 라이브러리

w, b, x를 동시에 학습하는 작업은 TensorFlow 신경망 패키지에서 제공하는 전형적인 '레이어'에 해당하지 않습니다. 
따라서 Course 2에서 사용한 Model, Compile(), Fit(), Predict() 흐름은 직접적으로 적용되지 않습니다. 
대신, 사용자 정의 학습 루프(custom training loop)를 사용할 수 있습니다.


반복 (수렴할 때까지):
- 순전파 계산 (compute forward pass)
- 손실(loss)에 대한 파라미터의 도함수 계산 (compute the derivatives of the loss relative to parameters)
- 학습률과 계산된 도함수를 사용하여 파라미터 업데이트 (update the parameters using the learning rate and the computed derivatives)

TensorFlow는 도함수를 계산하는 놀라운 기능을 가지고 있습니다. 
- tf.GradientTape() 섹션 내에서, TensorFlow 변수에 대한 연산이 추적됩니다.
- tape.gradient()가 호출되면, 이는 추적된 변수에 대한 손실의 그래디언트를 반환합니다. 
- 최적화를 사용하여 파라미터에 그래디언트를 적용할 수 있습니다.

In [16]:
import tensorflow as tf

In [17]:
w = tf.Variable(3.0) # 최적화 대상임을 알림
x=1.0
y=1.0
alpha= 0.01

iterations=30
for iter in range(iterations):
    # 계산 방법을 알려줌(자동미분 그래프)
    with tf.GradientTape() as tape:
        fwb= w*x
        costJ=(fwb-y)**2
    # 계산 호출(명시적으로 비용함수와 최적화 대상 전달)
    [dJdw] = tape.gradient(costJ, [w])
    # 명시적 업데이트 필요
    w.assign_add(-alpha*dJdw)

# Movie Example
- https://medium.com/@jigarvaghasiya5858/building-your-own-movie-recommender-system-a-hands-on-guide-to-collaborative-filtering-42d84b2aaac5 

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

In [20]:
#Load data
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443



<a name="4.1"></a>
### 4.1 Collaborative filtering cost function

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
The first summation in (1) is "for all $i$, $j$ where $r(i,j)$ equals $1$" and could be written:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularization}
$$

You should now write cofiCostFunc (collaborative filtering cost function) to return this cost.

In [25]:
# tensorflow 버전
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [26]:
# numpy 버전

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    Example:  
        Y (4778, 443) R (4778, 443)
        X (4778, 10)
        W (443, 10)
        b (1, 443)
        num_features 10
        num_movies 4778
        num_users 443
    """
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    pred_Y= np.matmul(X, W.T)+b
    error=(pred_Y-Y)**2
    loss= np.sum(error*R)/2
    
    regl_w= (lambda_*np.sum(W**2))/2
    regl_x= (lambda_*np.sum(X**2))/2
    
    J= loss+regl_w+regl_x
    ### END CODE HERE ### 

    return J

## 5 - Learning movie recommendations
아래의 셀에 당신의 영화 선택을 입력할 수 있습니다. 그러면 알고리즘이 당신을 위한 추천을 만들어줄 것입니다. 우리는 우리의 취향에 맞게 몇 가지 값을 채워 넣었지만, 당신이 우리의 선택으로 작동하는 것을 확인한 후에, 당신의 취향에 맞게 변경해야 합니다.

In [42]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)     

In [43]:
# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[2700] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

In [44]:
# Reload ratings
Y, R = load_ratings_small()

# Add new user ratings to Y 
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

In [45]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [46]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)
iterations = 200
lambda_ = 1
for iter in range(iterations):
    with tf.GradientTape() as tape:
        # Ynorm 평균정규화된 점수, R 점수 매김 여부
        cost_value = cofi_cost_func_v(X, W, b ,Ynorm, R, lambda_)
    grads= tape.gradient(cost_value,[X,W,b])
    # 미분값 적용 옵티마이저가~
    optimizer.apply_gradients(zip(grads,[X,W,b]))
    
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136169.3
Training loss at iteration 40: 51863.7
Training loss at iteration 60: 24599.0
Training loss at iteration 80: 13630.6
Training loss at iteration 100: 8487.7
Training loss at iteration 120: 5807.8
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.3
Training loss at iteration 180: 2902.1


<a name="6"></a>
## 6 - Recommendations

아래에서는 모든 영화와 사용자에 대한 평점을 계산하고, 추천되는 영화를 표시합니다. 
이는 위에서 my_ratings[]로 입력된 영화와 평점에 기반합니다. 
사용자𝑗가 영화𝑖에 대해 평점을 예측하려면,w(j)⋅x(i)+b(j)를 계산합니다. 

이는 행렬 곱셈을 사용하여 모든 평점에 대해 계산할 수 있습니다.

In [47]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic

In [48]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
1743,4.030961,4.252336,107,"Departed, The (2006)"
2112,3.985281,4.238255,149,"Dark Knight, The (2008)"
211,4.477798,4.122642,159,Memento (2000)
929,4.887054,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.796531,4.109091,55,Toy Story 3 (2010)
653,4.357304,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
1122,4.004471,4.006494,77,Shaun of the Dead (2004)
1841,3.980649,4.000000,61,Hot Fuzz (2007)
3083,4.084643,3.993421,76,"Dark Knight Rises, The (2012)"
2804,4.434171,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...
